In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy
import os
import os.path 
import matplotlib.pyplot as plt
import matplotlib.colors
import copy
import errno
import glob
import time
import calendar
import json
import pickle
import random
import netCDF4
from numpy import savez_compressed
from optparse import OptionParser
from scipy.interpolate import (UnivariateSpline, RectBivariateSpline, RegularGridInterpolator)
from sklearn.metrics import confusion_matrix

In [5]:
import sys
sys.path.append('../')
from src import utils, FogNet, FogNetConfig, cnn_evaluate, models, xai_engine

In [6]:
import warnings
warnings.filterwarnings('ignore')

## FogNet (24 Hr. 1600m) based on PSS, HSS, and CSS: 

In [7]:
Training_targets  = pd.read_csv(os.path.join('../trained_model/VIS_True_TRAIN.csv'))
Training_targets  = Training_targets[['CAT1600']].to_numpy().ravel()
Testing_targets      = pd.read_csv(os.path.join('../trained_model/VIS_True_TEST.csv'))
Testing_targets      = Testing_targets[['CAT1600']].to_numpy().ravel()

In [8]:
fognet_training_cat_prob  = pd.read_csv(os.path.join('../trained_model/VIS_Prob_TRAIN.csv'))
fognet_training_cat_prob  = fognet_training_cat_prob[['C0_Prob', 'C1_Prob']].to_numpy()
fognet_test_cat_prob      = pd.read_csv(os.path.join('../trained_model/VIS_Prob_TEST.csv'))
fognet_test_cat_prob      = fognet_test_cat_prob[['C0_Prob', 'C1_Prob']].to_numpy()

In [14]:
hss_training_threshold, hss_accuray_list, hss_Res241600  = cnn_evaluate.skilled_metrics(Training_targets, fognet_training_cat_prob, 'HSS')
hss_testing_accuracy                      = cnn_evaluate.test_eval(Testing_targets, fognet_test_cat_prob, hss_training_threshold)
pss_training_threshold, pss_accuray_list, pss_Res241600  = cnn_evaluate.skilled_metrics(Training_targets, fognet_training_cat_prob, 'PSS')
pss_testing_accuracy                      = cnn_evaluate.test_eval(Testing_targets, fognet_test_cat_prob, pss_training_threshold)
css_training_threshold, css_accuray_list, css_Res241600  = cnn_evaluate.skilled_metrics(Training_targets, fognet_training_cat_prob, 'CSS')
css_testing_accuracy                      = cnn_evaluate.test_eval(Testing_targets, fognet_test_cat_prob, css_training_threshold)

In [15]:
print(f"------  HSS  ------  PSS  ------  CSS")
print(f"POD:   {hss_testing_accuracy[4]:.2f}         {pss_testing_accuracy[4]:.2f}         {css_testing_accuracy[4]:.2f}")
print(f"F:     {hss_testing_accuracy[5]:.2f}         {pss_testing_accuracy[5]:.2f}         {css_testing_accuracy[5]:.2f}")
print(f"FAR:   {hss_testing_accuracy[6]:.2f}         {pss_testing_accuracy[6]:.2f}         {css_testing_accuracy[6]:.2f}")
print(f"CSI:   {hss_testing_accuracy[7]:.2f}         {pss_testing_accuracy[7]:.2f}         {css_testing_accuracy[7]:.2f}")
print(f"PSS:   {hss_testing_accuracy[8]:.2f}         {pss_testing_accuracy[8]:.2f}         {css_testing_accuracy[8]:.2f}")
print(f"HSS:   {hss_testing_accuracy[9]:.2f}         {pss_testing_accuracy[9]:.2f}         {css_testing_accuracy[9]:.2f}")
print(f"ORSS:  {hss_testing_accuracy[10]:.2f}         {pss_testing_accuracy[10]:.2f}         {css_testing_accuracy[10]:.2f}")
print(f"CSS:   {hss_testing_accuracy[11]:.2f}         {pss_testing_accuracy[11]:.2f}         {css_testing_accuracy[11]:.2f}")

------  HSS  ------  PSS  ------  CSS
POD:   0.55         0.58         0.28
F:     0.02         0.03         0.00
FAR:   0.49         0.61         0.32
CSI:   0.36         0.31         0.25
PSS:   0.54         0.55         0.28
HSS:   0.52         0.45         0.39
ORSS:  0.97         0.96         0.98
CSS:   0.50         0.38         0.66


### Permutation 

In [23]:
pfi_df = pd.read_csv('./PFIGW.csv')
permut_hss_results = pfi_df['HSS_Mean']

    
permutation_hss_results = [(0.52 - permut_hss_results[i]) for i in range(5)]
print("Permutation HSS based results:")
print("------  Score")
print(f"G1:    {permutation_hss_results[0] * 100:.2f}")
print(f"G2:    {permutation_hss_results[1] * 100:.2f}")
print(f"G3:    {permutation_hss_results[2] * 100:.2f}")
print(f"G4:    {permutation_hss_results[3] * 100:.2f}")
print(f"G5:    {permutation_hss_results[4] * 100:.2f}")

Permutation HSS based results:
------  Score
G1:    1.46
G2:    -0.14
G3:    0.67
G4:    -0.98
G5:    2.23


### Group Hold Hout

In [43]:
onehout_dir = '../trained_model/xai_trained_models/onehout/'
onehout_folder_list = sorted(os.listdir(onehout_dir))

onehout_hss_results = []

for i in range(1, len(onehout_folder_list)):
    y_training_cat_prob  = pd.read_csv(os.path.join(onehout_dir + onehout_folder_list[i] + '/VIS_Prob_TRAIN.csv'))
    y_training_cat_prob  = y_training_cat_prob[['C0_Prob', 'C1_Prob']].to_numpy()
    y_test_cat_prob      = pd.read_csv(os.path.join(onehout_dir + onehout_folder_list[i] + '/VIS_Prob_TEST.csv'))
    y_test_cat_prob      = y_test_cat_prob[['C0_Prob', 'C1_Prob']].to_numpy()
    
    
    training_threshold, accuray_list, Res   = cnn_evaluate.skilled_metrics(Training_targets, y_training_cat_prob, 'PSS')
    testing_accuracy =  cnn_evaluate.test_eval(Testing_targets, y_test_cat_prob, training_threshold)
    onehout_hss_results.append(testing_accuracy) 
onegout_hss_results = [(0.554324628592346 - onehout_hss_results[i][8]) for i in range(5)]
print("OneGHoldout HSS based results:")
print("------ Score")
print(f"G1:    {onegout_hss_results[0] * 100:.2f}")
print(f"G2:    {onegout_hss_results[1] * 100:.2f}")
print(f"G3:    {onegout_hss_results[2] * 100:.2f}")
print(f"G4:    {onegout_hss_results[3] * 100:.2f}")
print(f"G5:    {onegout_hss_results[4] * 100:.2f}")

OneGHoldout HSS based results:
------ Score
G1:    19.96
G2:    17.51
G3:    48.76
G4:    22.06
G5:    27.34


### LossSHAP

In [103]:
def LossSHAP_Scores(df, hss):
    
    Weights = [0.2, 0.2, 0.2, 0.2, 0.2, 
               0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 
               0.0333333,  0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 
               0.05, 0.05, 0.05, 0.05, 0.05,
               0.2] 

    G1_idx = [0, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20, 25, 26, 27, 28, 30]
    G1_weights = [Weights[index] for index in G1_idx]   
    G1_Sub = [(df.iloc[0]['HSS_Mean'] - hss), 
              (df.iloc[5]['HSS_Mean'] - df.iloc[1]['HSS_Mean']), 
              (df.iloc[6]['HSS_Mean'] - df.iloc[2]['HSS_Mean']), 
              (df.iloc[7]['HSS_Mean'] - df.iloc[3]['HSS_Mean']), 
              (df.iloc[8]['HSS_Mean'] - df.iloc[4]['HSS_Mean']), 
              (df.iloc[15]['HSS_Mean'] - df.iloc[9]['HSS_Mean']),  
              (df.iloc[16]['HSS_Mean'] - df.iloc[10]['HSS_Mean']), 
              (df.iloc[17]['HSS_Mean'] - df.iloc[11]['HSS_Mean']), 
              (df.iloc[18]['HSS_Mean'] - df.iloc[12]['HSS_Mean']), 
              (df.iloc[19]['HSS_Mean'] - df.iloc[13]['HSS_Mean']), 
              (df.iloc[20]['HSS_Mean'] - df.iloc[14]['HSS_Mean']), 
              (df.iloc[25]['HSS_Mean'] - df.iloc[21]['HSS_Mean']),
              (df.iloc[26]['HSS_Mean'] - df.iloc[22]['HSS_Mean']), 
              (df.iloc[27]['HSS_Mean'] - df.iloc[23]['HSS_Mean']), 
              (df.iloc[28]['HSS_Mean'] - df.iloc[24]['HSS_Mean']), 
              (hss - df.iloc[29]['HSS_Mean'])]
    G1_score = sum([a*b for a, b in zip(G1_weights, G1_Sub)])
    
    G2_idx = [1, 5, 9, 10, 11, 15, 16, 17, 21, 22, 23, 25, 26, 27, 29, 30]
    G2_weights = [Weights[index] for index in G2_idx]
    G2_Sub = [(df.iloc[1]['HSS_Mean'] - hss), 
              (df.iloc[5]['HSS_Mean'] - df.iloc[0]['HSS_Mean']), 
              (df.iloc[9]['HSS_Mean'] - df.iloc[2]['HSS_Mean']), 
              (df.iloc[10]['HSS_Mean'] - df.iloc[3]['HSS_Mean']), 
              (df.iloc[11]['HSS_Mean'] - df.iloc[4]['HSS_Mean']), 
              (df.iloc[15]['HSS_Mean'] - df.iloc[6]['HSS_Mean']),  
              (df.iloc[16]['HSS_Mean'] - df.iloc[7]['HSS_Mean']), 
              (df.iloc[17]['HSS_Mean'] - df.iloc[8]['HSS_Mean']), 
              (df.iloc[21]['HSS_Mean'] - df.iloc[12]['HSS_Mean']), 
              (df.iloc[22]['HSS_Mean'] - df.iloc[13]['HSS_Mean']), 
              (df.iloc[23]['HSS_Mean'] - df.iloc[14]['HSS_Mean']), 
              (df.iloc[25]['HSS_Mean'] - df.iloc[18]['HSS_Mean']),
              (df.iloc[26]['HSS_Mean'] - df.iloc[19]['HSS_Mean']), 
              (df.iloc[27]['HSS_Mean'] - df.iloc[20]['HSS_Mean']), 
              (df.iloc[29]['HSS_Mean'] - df.iloc[24]['HSS_Mean']), 
              (hss - df.iloc[28]['HSS_Mean'])]
    G2_score = sum([a*b for a, b in zip(G2_weights, G2_Sub)])
 
    G3_idx = [2, 6, 9, 12, 13, 15, 18, 19, 21, 22, 24, 25, 26, 28, 29, 30]
    G3_weights = [Weights[index] for index in G3_idx]
    G3_Sub = [(df.iloc[2]['HSS_Mean'] - hss), 
              (df.iloc[6]['HSS_Mean'] - df.iloc[0]['HSS_Mean']), 
              (df.iloc[9]['HSS_Mean'] - df.iloc[1]['HSS_Mean']), 
              (df.iloc[12]['HSS_Mean'] - df.iloc[3]['HSS_Mean']), 
              (df.iloc[13]['HSS_Mean'] - df.iloc[4]['HSS_Mean']), 
              (df.iloc[15]['HSS_Mean'] - df.iloc[5]['HSS_Mean']),  
              (df.iloc[18]['HSS_Mean'] - df.iloc[7]['HSS_Mean']), 
              (df.iloc[19]['HSS_Mean'] - df.iloc[8]['HSS_Mean']), 
              (df.iloc[21]['HSS_Mean'] - df.iloc[10]['HSS_Mean']), 
              (df.iloc[22]['HSS_Mean'] - df.iloc[11]['HSS_Mean']), 
              (df.iloc[24]['HSS_Mean'] - df.iloc[14]['HSS_Mean']), 
              (df.iloc[25]['HSS_Mean'] - df.iloc[16]['HSS_Mean']),
              (df.iloc[26]['HSS_Mean'] - df.iloc[17]['HSS_Mean']), 
              (df.iloc[28]['HSS_Mean'] - df.iloc[20]['HSS_Mean']), 
              (df.iloc[29]['HSS_Mean'] - df.iloc[23]['HSS_Mean']), 
              (hss - df.iloc[27]['HSS_Mean'])]
    G3_score = sum([a*b for a, b in zip(G3_weights, G3_Sub)])

    G4_idx = [3, 7, 10, 12, 14, 16, 18, 20, 21, 23, 24, 26, 27, 28, 29, 30]
    G4_weights = [Weights[index] for index in G4_idx]
    G4_Sub = [(df.iloc[3]['HSS_Mean'] - hss), 
              (df.iloc[7]['HSS_Mean'] - df.iloc[0]['HSS_Mean']), 
              (df.iloc[10]['HSS_Mean'] - df.iloc[1]['HSS_Mean']), 
              (df.iloc[12]['HSS_Mean'] - df.iloc[2]['HSS_Mean']), 
              (df.iloc[14]['HSS_Mean'] - df.iloc[4]['HSS_Mean']), 
              (df.iloc[16]['HSS_Mean'] - df.iloc[5]['HSS_Mean']),  
              (df.iloc[18]['HSS_Mean'] - df.iloc[6]['HSS_Mean']), 
              (df.iloc[20]['HSS_Mean'] - df.iloc[8]['HSS_Mean']), 
              (df.iloc[21]['HSS_Mean'] - df.iloc[9]['HSS_Mean']), 
              (df.iloc[23]['HSS_Mean'] - df.iloc[11]['HSS_Mean']), 
              (df.iloc[24]['HSS_Mean'] - df.iloc[13]['HSS_Mean']), 
              (df.iloc[25]['HSS_Mean'] - df.iloc[15]['HSS_Mean']),
              (df.iloc[27]['HSS_Mean'] - df.iloc[17]['HSS_Mean']), 
              (df.iloc[28]['HSS_Mean'] - df.iloc[19]['HSS_Mean']), 
              (df.iloc[29]['HSS_Mean'] - df.iloc[22]['HSS_Mean']), 
              (hss - df.iloc[26]['HSS_Mean'])]
    G4_score = sum([a*b for a, b in zip(G4_weights, G4_Sub)])

    G5_idx     = [4, 8, 11, 13, 14, 17, 19, 20, 22, 23, 24, 26, 27, 28, 29, 30] 
    G5_weights = [Weights[index] for index in G5_idx]
    G5_Sub = [(df.iloc[4]['HSS_Mean'] - hss), 
              (df.iloc[8]['HSS_Mean'] - df.iloc[0]['HSS_Mean']), 
              (df.iloc[11]['HSS_Mean'] - df.iloc[1]['HSS_Mean']), 
              (df.iloc[13]['HSS_Mean'] - df.iloc[2]['HSS_Mean']), 
              (df.iloc[14]['HSS_Mean'] - df.iloc[3]['HSS_Mean']), 
              (df.iloc[17]['HSS_Mean'] - df.iloc[5]['HSS_Mean']),  
              (df.iloc[19]['HSS_Mean'] - df.iloc[6]['HSS_Mean']), 
              (df.iloc[20]['HSS_Mean'] - df.iloc[7]['HSS_Mean']), 
              (df.iloc[22]['HSS_Mean'] - df.iloc[9]['HSS_Mean']), 
              (df.iloc[23]['HSS_Mean'] - df.iloc[10]['HSS_Mean']), 
              (df.iloc[24]['HSS_Mean'] - df.iloc[12]['HSS_Mean']), 
              (df.iloc[26]['HSS_Mean'] - df.iloc[15]['HSS_Mean']),
              (df.iloc[27]['HSS_Mean'] - df.iloc[16]['HSS_Mean']), 
              (df.iloc[28]['HSS_Mean'] - df.iloc[18]['HSS_Mean']), 
              (df.iloc[29]['HSS_Mean'] - df.iloc[21]['HSS_Mean']), 
              (hss - df.iloc[25]['HSS_Mean'])]

    G5_score = sum([a*b for a, b in zip(G5_weights, G5_Sub)])
    
    
    return G1_score, G2_score, G3_score, G4_score, G5_score

In [104]:
LossSHAP_df = pd.read_csv('./LossSHAP.csv')

G1_score, G2_score, G3_score, G4_score, G5_score = LossSHAP_Scores(LossSHAP_df, 0.52)

In [105]:
print("LossSHAP HSS based results:")
print("------ Score")
print(f"G1:    {G1_score * 100:.2f}")
print(f"G2:    {G2_score * 100:.2f}")
print(f"G3:    {G3_score * 100:.2f}")
print(f"G4:    {G4_score * 100:.2f}")
print(f"G5:    {G5_score * 100:.2f}")

LossSHAP HSS based results:
------ Score
G1:    -9.39
G2:    -4.19
G3:    -6.33
G4:    14.16
G5:    5.75
